<a href="https://colab.research.google.com/github/Napster2417a/Maps-ProblemaViajante/blob/main/Busquedas_Maps.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install googlemaps haversine

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install ortools

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 30.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.4/302.4 kB 18.8 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.20.3
    Uninstalling protobuf-3.20.3:
      Successfully uninstalled protobuf-3.20.3


In [ ]:
!pip install tabulate


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import googlemaps
from haversine import haversine

from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp


API_KEY = 'Tu API'
gmaps = googlemaps.Client(key=API_KEY)

moncloa_address = 'Moncloa, Madrid'
moncloa_location = gmaps.geocode(moncloa_address)[0]['geometry']['location']
moncloa_coordinates = (moncloa_location['lat'], moncloa_location['lng'])

madrid_center_address = 'Puerta del Sol, Madrid'
madrid_center_location = gmaps.geocode(madrid_center_address)[0]['geometry']['location']
madrid_radius = 20000  # 20 km

def get_workshops_nearby(center_location, radius):
    places_result = gmaps.places_nearby(location=center_location, radius=radius, type='car_repair')
    workshops = []

    for place in places_result['results']:
        name = place['name']
        lat = place['geometry']['location']['lat']
        lng = place['geometry']['location']['lng']
        distance = haversine(moncloa_coordinates, (lat, lng))
        place_id = place['place_id']

        workshops.append({
            'name': name,
            'location': (lat, lng),
            'distance': distance,
            'place_id': place_id
        })

    return workshops

def filter_car_workshops(workshops):
    keywords = ['moto', 'motocicleta', 'motorcycle', 'bike']
    car_workshops = []

    for workshop in workshops:
        place_details = gmaps.place(workshop['place_id'])
        exclude = False

        for keyword in keywords:
            if keyword.lower() in place_details['result']['name'].lower() or (place_details['result'].get('formatted_address') and keyword.lower() in place_details['result']['formatted_address'].lower()) or (place_details['result'].get('vicinity') and keyword.lower() in place_details['result']['vicinity'].lower()):
                exclude = True
                break

        if not exclude:
            car_workshops.append(workshop)

    return car_workshops


def create_distance_matrix(workshops):
    matrix = []
    for origin in workshops:
        row = []
        for destination in workshops:
            distance = haversine(origin['location'], destination['location'])
            row.append(distance)
        matrix.append(row)
    return matrix

def tsp_solution(workshops):
    distance_matrix = create_distance_matrix(workshops)

    manager = pywrapcp.RoutingIndexManager(len(distance_matrix), 1, 0)
    routing = pywrapcp.RoutingModel(manager)

    def distance_callback(from_index, to_index):
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        return distance_matrix[from_node][to_node]

    transit_callback_index = routing.RegisterTransitCallback(distance_callback)
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.first_solution_strategy = (routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)

    solution = routing.SolveWithParameters(search_parameters)
    return solution, manager, routing

def print_solution(solution, manager, routing, workshops):
    vehicle_index = 0
    index = manager.GetStartIndex(vehicle_index)
    route = []
    while not routing.IsEnd(index):
        node_index = manager.IndexToNode(index)
        route.append(node_index)
        index = solution.Value(routing.NextVar(index))
    route.append(manager.IndexToNode(index))

    total_distance = solution.ObjectiveValue()

    print(f"Recorrido total: {total_distance} km")
    for i, index in enumerate(route):
        workshop = workshops[index]
        lat, lng = workshop['location']
        address = gmaps.reverse_geocode((lat, lng))[0]['formatted_address']
        print(f"{i+1}. {workshop['name']} - {address} - {workshop['distance']} km")


workshops = get_workshops_nearby(madrid_center_location, madrid_radius)
car_workshops = filter_car_workshops(workshops)
sorted_workshops = sorted(car_workshops, key=lambda x: x['distance'])



workshops.insert(0, {
    'name': 'Moncloa',
    'location': moncloa_coordinates,
    'distance': 0
})

solution, manager, routing = tsp_solution(sorted_workshops)
print_solution(solution, manager, routing, sorted_workshops)



Recorrido total: 1 km
1. STS Auto. Protección del automóvil - C. de Arriaza, 11, 28008 Madrid, Spain - 3.8704394378318963 km
2. B y S Técnicas Manutención Mecánica, S.L. - Cuesta de Sto. Domingo, 20, 28013 Madrid, Spain - 4.535658801492175 km
3. Repuestos Garcia - Prta del Sol, 10, 28013 Madrid, Spain - 4.994814425152899 km
4. Vespa Roma - C. de Piamonte, 2, 28004 Madrid, Spain - 5.258069987452674 km
5. Euromaster Madrid Chamberi - Calle de Covarrubias, 26, 28010 Madrid, Spain - 4.906573988189931 km
6. Rscross - C. de Valverde, 35, 28004 Madrid, Spain - 4.867625847271145 km
7. Auto Res - C. de la Salud, 19, 28013 Madrid, Spain - 4.933899865281315 km
8. Recuperaciones Cifuentes - C. de Colmenares, 3, 28004 Madrid, Spain - 5.347542775133569 km
9. Garaje Esperanza - Pje. Doré, 60, 28012 Madrid, Spain - 5.716313566900629 km
10. Manuel Peral Tejero - Rda. de Valencia, 4, 28012 Madrid, Spain - 6.036538657902187 km
11. Emilio Pos Electromechanics - C. de Cdad. Real, 20, 28045 Madrid, Spain - 